In [ ]:
import pandas as pd
import numpy as np
import itertools 
import matplotlib.pyplot as plt

In [ ]:
users = pd.read_csv(
    'Assignment_1/ml-1m/users.dat',
    sep = "::",
    names = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    encoding='latin-1',
    engine='python',
)

users.head()

In [ ]:
movies = pd.read_csv(
    'Assignment_1/ml-1m/movies.dat',
    sep = "::",
    names = ['MovieID', 'Title', 'Genres'],
    encoding='latin-1',
    engine='python',
)

movies.head()

In [ ]:
ratings = pd.read_csv(
    'Assignment_1/ml-1m/ratings.dat',
    sep = "::",
    names = ['UserID', 'MovieID', 'Rating', 'Timestamp'],
    encoding='latin-1',
    engine='python',
)

ratings.head()

In [ ]:
#MATRIX FACTORIZATION WITHOUT CROSS VALIDATION

learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

x = ratings.pivot(index='MovieID', columns='UserID', values='Rating')
M=np.random.rand(x.shape[0],num_factors)
M=pd.DataFrame(M, index=x.index)
U=np.random.rand(num_factors,x.shape[1])
U=pd.DataFrame(U, columns=x.columns)

indexes=x[x.notnull()].stack().index
iterat=1
rmse=100
newrmse=99

while (iterat < num_iter and newrmse-rmse < 0.01 ):
    print('iteration: ' + str(iterat))
    iterat += 1
    for k in indexes:
        movie = M.loc[k[0],]
        user = U.loc[:,k[1]]
        x_hat = np.dot(movie,user)
        error = x.loc[k[0],k[1]] - x_hat
        M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie)
        U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user)
            
    factoriz=pd.DataFrame(np.matmul(M, U), index=x.index, columns= x.columns)
    rmse=newrmse
    newrmse=np.sqrt(((factoriz-x)**2).mean().mean())
            
    #rmse = np.sqrt(np.mean((np.matmul(M, U)-train)**2))
mae = abs(factoriz-x).mean().mean()


In [ ]:
#MATRIX FACTORIZATION
# google colab / runtime - cambia tipo - GPU/TPU
x = ratings.sample(frac = 1)
X = np.array_split(x, 5)
learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

rmses = []
maes = []

for i in range(1):
    X_test = X[i]
    X_train = pd.concat([X[j] for j in range(5) if j!=i])
    train=X_train.pivot(index='MovieID', columns='UserID', values='Rating')
    test = X_test.pivot(index='MovieID', columns='UserID', values='Rating')
    
    M=np.random.rand(train.shape[0],num_factors)
    M=pd.DataFrame(M, index=train.index)
    U=np.random.rand(num_factors,train.shape[1])
    U=pd.DataFrame(U, columns=train.columns)
    
    indexes=train[train.notnull()].stack().index
    iterat=1
    rmse=100
    newrmse=99
    
    while (iterat < num_iter and newrmse-rmse < 0.01 ):
        print('Set: ' + str(i) + ', iteration: ' + str(iterat))
        iterat += 1
        for k in indexes:
            movie = M.loc[k[0],]
            user = U.loc[:,k[1]]
            x_hat = np.dot(movie,user)
            error = train.loc[k[0],k[1]] - x_hat
            M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie)
            U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user)
            
        factoriz=pd.DataFrame(np.matmul(M, U), index=train.index, columns= train.columns)
        rmse=newrmse
        newrmse=np.sqrt(((factoriz[factoriz.columns & test.columns]-test)**2).mean().mean())
            
    #rmse = np.sqrt(np.mean((np.matmul(M, U)-train)**2))
    rmses += [newrmse]
    mae = abs(factoriz[factoriz.columns & test.columns]-test).mean().mean()
    maes += [mae]

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Start by visualizing users
pca = PCA(n_components=2)

PCs = pca.fit_transform(U)


In [ ]:
# Visualize users labelled by genders
plt.figure(figsize=(10,10)) 

for gender in ['M', 'F']:
    mask = users[['Gender']] == gender
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=gender, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize users labelled by age
plt.figure(figsize=(10,10)) 
ages = np.unique(users['Age'])

for age in ages:
    mask = users[['Age']] == age
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=age, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by genre

### UMAP

In [ ]:
import umap

In [ ]:
umap = umap.UMAP()
X_u = umap.fit_transform(train_in)

In [ ]:
plt.figure(figsize=(10,10))

for i in range(10):
    mask = train_out == i
    plt.scatter(X_u[mask, 0], X_u[mask, 1], label=i, s=25, alpha=1)
plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', fontsize=20)

### t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
t_sne = TSNE(n_components=2)
X_tsne = t_sne.fit_transform(train_in)

In [ ]:
plt.figure(figsize=(10,10))

for i in range(10):
    mask = train_out == i
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], label=i, s=25, alpha=1)
plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', fontsize=20)

## 1.3

In [ ]:
Class = []
for i in train_in:
    Dis = []
    for j in range(10):
        Mean = means[j]
        dis = np.linalg.norm(Mean-i)
        Dis.append(dis)
    dis_min = np.amin(Dis)
    clas = float(np.where(Dis==dis_min)[0])
    Class.append(clas)

In [ ]:
Class = np.array(Class)
diff = train_out-Class

In [ ]:
success_rate = len(np.where(diff==0)[0])/len(train_out)
print(success_rate)

In [ ]:
test_in = pd.read_csv("test_in.csv")
test_out = pd.read_csv("test_out.csv")
test_in = np.array(test_in)
test_out = np.array(test_out)

In [ ]:
test_out = np.squeeze(test_out, axis=1)

In [ ]:
Class_test = []
for i in test_in:
    Dis = []
    for j in range(10):
        Mean = means[j]
        dis = np.linalg.norm(Mean-i)
        Dis.append(dis)
    dis_min = np.amin(Dis)
    clas = float(np.where(Dis==dis_min)[0])
    Class_test.append(clas)

In [ ]:
Class_test = np.array(Class_test)
diff_test = test_out - Class_test

In [ ]:
success_rate_test = len(np.where(diff_test==0)[0])/len(test_out)
print(success_rate_test)

## 1.4

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k = len(train_in)/10

In [ ]:
knn = KNeighborsClassifier(n_neighbors=12)

In [ ]:
knn.fit(train_in, train_out)
knn_out = knn.predict(test_in)

In [ ]:
diff_knn = knn_out - test_out
success_rate_test = len(np.where(diff_knn==0)[0])/len(test_out)
print(success_rate_test)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_m_knn = confusion_matrix(test_out, knn_out)

In [ ]:
conf_m_simp = confusion_matrix(test_out, Class_test)

In [ ]:
print(conf_m_knn)
print(conf_m_simp)